In [9]:
import json
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm

sys.path.append(os.path.abspath(".."))

from config import (
    FILTERED_DATA_PATH,
    TRAIN_DATA_PATH,
    VAL_DATA_PATH,
    TEST_DATA_PATH,
    METADATA_PATH,
    EMBEDDINGS_TITLE_PATH,
    EMBEDDINGS_DESCR_PATH,
    EMBEDDINGS_COMBI_PATH,
    N_COMPONENTS,
    RANDOM_STATE,
    N_RECOMMENDATIONS,
    N_EPOCHS,
    EPSILON,
    RECOMMENDATIONS_PATH,
    TRANSFORMED_DATA_PATH,
    SPLIT_DATE_TRAIN_VAL,
    SPLIT_DATE_VAL_TEST,
)
import utils

In [10]:
# importing training data
train_df = pd.read_parquet("../" + TRAIN_DATA_PATH)

# importing validation data
val_df = pd.read_parquet("../" + VAL_DATA_PATH)

# importing test data
test_df = pd.read_parquet("../" + TEST_DATA_PATH)

# importing metadata
meta_df = pd.read_parquet("../" + METADATA_PATH)

# importing the transformed data
transformed_df = pd.read_parquet("../" + TRANSFORMED_DATA_PATH)

# importing the filtered data
filtered_df = pd.read_parquet("../" + FILTERED_DATA_PATH)

In [12]:
# loading embeddings
emb_df = pd.read_parquet("../" + EMBEDDINGS_TITLE_PATH)
emb_df.drop(columns="episode", inplace=True)
min_value = emb_df.min().min()
max_value = emb_df.max().max()

print(f"Minimum value: {min_value}")
print(f"Maximum value: {max_value}")

Minimum value: -0.3516335799441893
Maximum value: 0.6239668024905588


In [13]:
# loading embeddings
emb_df = pd.read_parquet("../" + EMBEDDINGS_DESCR_PATH)
emb_df.drop(columns="episode", inplace=True)
min_value = emb_df.min().min()
max_value = emb_df.max().max()

print(f"Minimum value: {min_value}")
print(f"Maximum value: {max_value}")

Minimum value: -0.3516335799441893
Maximum value: 0.6149092034145577


In [14]:
# loading embeddings
emb_df = pd.read_parquet("../" + EMBEDDINGS_COMBI_PATH)
emb_df.drop(columns="episode", inplace=True)
min_value = emb_df.min().min()
max_value = emb_df.max().max()

print(f"Minimum value: {min_value}")
print(f"Maximum value: {max_value}")

Minimum value: -0.3516335799441893
Maximum value: 0.6149092034145577
